In [1]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


In [2]:
import matplotlib.pyplot as plt
import torch
from sdm.models import S2Dataset

In [3]:
input_vars = [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14
    ]
n_input_bands = len(input_vars)

train_dataset = S2Dataset(
    "data/processed/s2-lidar/s2-lidar-stack.tif",
    input_bands=input_vars,
    target_band=[15],
    tile_size=1024,
)

In [4]:
train_dataset[0]

/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/rasterio/features.py:324: ShapeSkipWarning: Invalid or empty shape -1.8374140821380693 at index 0 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/rasterio/features.py:324: ShapeSkipWarning: Invalid or empty shape 53.29282372497585 at index 1 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/rasterio/features.py:324: ShapeSkipWarning: Invalid or empty shape 10238.162585917862 at index 2 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/p

ValueError: No valid geometry objects found for rasterize

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# set mac m1 device
device = torch.device('mps')

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, in_channels):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 256 * 256, 256)  # Adjust the size depending on your tile size
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x




train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Initialize the model, loss, and optimizer
model = SimpleCNN(n_input_bands)
criterion = nn.MSELoss()  # Mean Squared Error for regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 10  # Number of epochs
for epoch in range(1, n_epochs + 1):
    epoch_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch}/{n_epochs} Loss: {epoch_loss/len(train_loader)}")

# Save the model
# torch.save(model.state_dict(), "simple_cnn_model.pth")


Error: No data found in bounds.
Error: No data found in bounds.
Error: No data found in bounds.
Error: No data found in bounds.


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>